<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<div align='center'>
###  Felipe González - 201273534-3 
### Ignacio Tampe - 201573514-k 
</div>
<hr style="height:2px;border:none"/>

## 1. Back-propagation (BP) from *Scratch*


In [21]:
import numpy as np #se importa numpy como np

In [72]:
class RedNeuronal:
    # num_entrada: dimension de los vectores de entrada, en iris: 4
    # tam_capas: lista de cantidad de neuronas por cada capa interna, en iris: [32, 16]
    # num_salida: dimension de los vectores de salida, en iris: 3
    def __init__(self, num_entrada, tam_capas, num_salida):
        self.num_entrada = num_entrada
        self.tam_capas = tam_capas
        self.num_salida = num_salida
        self.capas = []
        # iniciar todas las capas inicial - ocultas - final
        self.capas.append(CapaNeuronal(num_entrada))
        for i in range(1, len(tam_capas)):
            self.capas.append(CapaNeuronal(tam_capas[i-1], self.capas[i-1].tam_capa))
        self.capas.append(CapaNeuronal(num_salida, self.capas[len(self.capas)-1].tam_capa))
        
    def train(self, entrada, output, ciclos, tasa):
        # repetir para todas 
        for ciclo in range(ciclos):
            for vector in entrada:
                self.capas[0].set_capa(vector) #Poblar capa inicial
                # Feed forward
                for i in range(1, len(self.capas)):
                    self.capas[i].feed_forward(self.capas[i-1])
                
                self.capas[len(self.capas)-1].mostrar_capa()
                
                # evaluar error

                # Back propagation (actualizar pesos de las neuronas)
            
            



In [76]:
class CapaNeuronal:
    def __init__(self, tam_capa, tam_anterior = 0):
        self.tam_capa = tam_capa
        self.bias = 1
        self.neuronas = [Neurona(tam_anterior)] * tam_capa # lista de neuronas de la capa
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def get_activaciones(self):
        return [neurona.activacion for neurona in self.neuronas]
    
    def feed_forward(self, capa_anterior):
        for neurona in self.neuronas:
            activaciones = capa_anterior.get_activaciones()
            pesos = neurona.pesos
            result = self.sigmoid(np.dot(pesos, activaciones) + self.bias)          
            neurona.set_activacion(result)
            
    #Funcion para poblar datos de entrada a la red.
    def set_capa(self, valores):
        for i in range(len(self.neuronas)):
            self.neuronas[i].set_activacion(valores[i])
            
    def mostrar_capa(self):
        for neurona in self.neuronas:
            print(neurona, end="\t")
        print("\n-----------------------------")

        
class Neurona:
    def __init__(self, tam_capa_anterior):
        self.pesos = np.random.rand(tam_capa_anterior) # genera una lista de numeros aleatorios de tamaño "tam_capa_anterior"
        
    def set_activacion(self, valor):
        self.activacion = valor
        
    def __str__(self):
        return "Activación: "+str(self.activacion)

### ¡Es hora de probar!

In [77]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
X_train,y_train = load_iris(return_X_y=True)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
#transform target to one hot vector
nn = RedNeuronal(4, [32, 16], 3)
nn.train(X_train, y_train, 1, 0.1)

Activación: 0.935963209248	Activación: 0.935963209248	Activación: 0.935963209248	
-----------------------------
Activación: 0.935963209248	Activación: 0.935963209248	Activación: 0.935963209248	
-----------------------------
Activación: 0.935963209248	Activación: 0.935963209248	Activación: 0.935963209248	
-----------------------------
Activación: 0.935963209248	Activación: 0.935963209248	Activación: 0.935963209248	
-----------------------------
Activación: 0.935963209248	Activación: 0.935963209248	Activación: 0.935963209248	
-----------------------------
Activación: 0.97878743131	Activación: 0.97878743131	Activación: 0.97878743131	
-----------------------------
Activación: 0.960662602894	Activación: 0.960662602894	Activación: 0.960662602894	
-----------------------------
Activación: 0.935963209248	Activación: 0.935963209248	Activación: 0.935963209248	
-----------------------------
Activación: 0.935963209248	Activación: 0.935963209248	Activación: 0.935963209248	
-------------------------

## 1. Referencias


In [18]:
import keras
y_onehot = keras.utils.to_categorical(y_train)

[[ -9.00681170e-01   1.03205722e+00  -1.34127240e+00  -1.31297673e+00]
 [ -1.14301691e+00  -1.24957601e-01  -1.34127240e+00  -1.31297673e+00]
 [ -1.38535265e+00   3.37848329e-01  -1.39813811e+00  -1.31297673e+00]
 [ -1.50652052e+00   1.06445364e-01  -1.28440670e+00  -1.31297673e+00]
 [ -1.02184904e+00   1.26346019e+00  -1.34127240e+00  -1.31297673e+00]
 [ -5.37177559e-01   1.95766909e+00  -1.17067529e+00  -1.05003079e+00]
 [ -1.50652052e+00   8.00654259e-01  -1.34127240e+00  -1.18150376e+00]
 [ -1.02184904e+00   8.00654259e-01  -1.28440670e+00  -1.31297673e+00]
 [ -1.74885626e+00  -3.56360566e-01  -1.34127240e+00  -1.31297673e+00]
 [ -1.14301691e+00   1.06445364e-01  -1.28440670e+00  -1.44444970e+00]
 [ -5.37177559e-01   1.49486315e+00  -1.28440670e+00  -1.31297673e+00]
 [ -1.26418478e+00   8.00654259e-01  -1.22754100e+00  -1.31297673e+00]
 [ -1.26418478e+00  -1.24957601e-01  -1.34127240e+00  -1.44444970e+00]
 [ -1.87002413e+00  -1.24957601e-01  -1.51186952e+00  -1.44444970e+00]
 [ -5.

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8253083999358165994
]


### Pregunta 2

In [14]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer=SGD(lr=0.1),loss="mse", metrics=["accuracy"])

In [20]:
model.fit(X_train, y_onehot, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0100 - acc: 0.9867
Epoch 2/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0094 - acc: 0.9867
Epoch 3/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0094 - acc: 0.9867
Epoch 4/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0113 - acc: 0.9800A: 0s - loss: 0.0187 - acc: 0.9
Epoch 5/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0085 - acc: 0.9800- ETA: 0s - loss: 0.0067 - acc: 0.9773 
Epoch 6/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0105 - acc: 0.9800
Epoch 7/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0095 - acc: 0.9800
Epoch 8/100
150/150 [==============================] - 0s 2ms/step - loss: 0.0111 - acc: 0.9800
Epoch 9/100
150/150 [==============================] - 0s 1ms/step - loss: 0.0100 - acc: 0.9800
Epoch 10/100
150/150 [==============================] - 0s 1ms/ste